For a more complex example, here's how to calculate thermal transmittance considering thermal bridges in a heterogeneous element:

In [ ]:
def calculate_heterogeneous_rt(layers_and_fractions):
    """
    Calculate resistance of heterogeneous construction (upper and lower limits)

    Parameters:
    layers_and_fractions: list of layers, where each layer is
                         either a single value (homogeneous layer resistance)
                         or a list of tuples (resistance, area_fraction) for heterogeneous layers

    Returns:
    RT: Average thermal resistance
    """
    # Calculate upper limit (R'T)
    total_r_upper = 0

    for layer in layers_and_fractions:
        if isinstance(layer, (int, float)):
            # Homogeneous layer
            total_r_upper += layer
        else:
            # Heterogeneous layer - calculate parallel resistances
            layer_r_upper = 0
            for r, fraction in layer:
                layer_r_upper += fraction / r
            total_r_upper += 1 / layer_r_upper

    # Calculate lower limit (R''T)
    total_r_lower = 0

    # Transpose the data structure to process by column
    n_columns = len([item for item in layers_and_fractions if isinstance(item, list)][0])
    columns = [0] * n_columns

    for i, layer in enumerate(layers_and_fractions):
        if isinstance(layer, (int, float)):
            # Add homogeneous resistance to all columns
            for j in range(n_columns):
                columns[j] += layer
        else:
            # Add heterogeneous resistance to corresponding column
            for j, (r, _) in enumerate(layer):
                columns[j] += r

    # Calculate total for each column
    for j in range(n_columns):
        r_column = columns[j]
        _, fraction = [item for item in layers_and_fractions if isinstance(item, list)][0][j]
        total_r_lower += fraction / r_column

    # Average the limits
    RT = (total_r_upper + 1/total_r_lower) / 2

    return RT

In [ ]:
# Example: Wall with thermal bridges (simplified for demonstration)
# Structure:
# - Exterior surface resistance: 0.04 m²K/W
# - Brick (homogeneous): R=0.21 m²K/W
# - Insulation with wooden studs:
#   80% insulation (R=1.61 m²K/W) and 20% wood studs (R=0.45 m²K/W)
# - Gypsum board (homogeneous): R=0.06 m²K/W
# - Interior surface resistance: 0.13 m²K/W

wall_layers = [
    0.04,  # Rse
    0.21,  # Brick
    [(1.61, 0.8), (0.45, 0.2)],  # Insulation with wood studs
    0.06,  # Gypsum board
    0.13   # Rsi
]

rt_wall = calculate_heterogeneous_rt(wall_layers)
u_wall = 1 / rt_wall

print(f"Wall with thermal bridges U-value: {u_wall:.3f} W/m²K")